In [1]:
from pynq import Device, Overlay
ol = Overlay("bitstreams/u280_xclbin/500_250_HBM/FullW.xclbin",download=False,device=Device.devices[0])
print("Overlay downloaded.")

Overlay downloaded.


In [25]:
ol.is_loaded()

True

In [24]:
ol.timestamp = None

In [13]:
ol.bitfile_name

'/mnt/scratch/ldierick/octoray/fwi/bitstreams/u280_xclbin/500_250_HBM/FullW.xclbin'

In [22]:
print(ol.device.timestamp)

None


In [16]:
ol.device.bitfile_name = ol.bitfile_name

AttributeError: can't set attribute

In [4]:
dir(ol._ip_map)

['HBM0',
 'HBM1',
 'HBM10',
 'HBM11',
 'HBM2',
 'HBM3',
 'HBM4',
 'HBM5',
 'HBM6',
 'HBM7',
 'HBM8',
 'HBM9',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_description',
 '_keys',
 'dotprod_1',
 'dotprod_2',
 'update_1',
 'update_2']

In [26]:
from pynq import allocate 
import numpy as np

A = allocate(shape=(500,250), dtype=np.complex64, target=getattr(ol,"HBM3"))

In [27]:
cu = 0
resolution = 500
gridsize = 250
ROW=500
COL=250

def init_hw_dotProduct(s_in_V,s2_in_V):       
    COL = 250
    ROW = 500
    for col in range(COL):
        s2_in_V[col] = col * 1.0
        for row in range(ROW):
            s_in_V[row][col] = complex(row*1.0,row*col*0.33)
            
    return s_in_V, s2_in_V

def init_hw_updateDirection(resV_V,kappa_V):       
    for row in range(ROW):
        resV_V[row] = complex(row*1.0,row*0.33)
        for col in range(COL):
            kappa_V[row][col] = complex(row*1.0,row*col*0.33)
    return resV_V, kappa_V

dotprod = ol.dotprod_2
update = ol.update_2

B = allocate(shape=(gridsize,), dtype=np.float32, target=getattr(ol,"HBM4"))
C = allocate(shape=(resolution,), dtype=np.complex64, target=getattr(ol,"HBM5"))

D = allocate(shape=(resolution,gridsize), dtype=np.complex64,  target=getattr(ol,"HBM9"))
E = allocate(shape=(resolution,),dtype=np.complex64,  target=getattr(ol,"HBM10"))
F = allocate(shape=(gridsize), dtype=np.complex64, target=getattr(ol,"HBM11"))

   
A, B = init_hw_dotProduct(A, B)
E, D = init_hw_updateDirection(E,D)

print(f'start dotprod_{cu}')
A.sync_to_device()
B.sync_to_device()
dotprod.call(A,B,C)   
C.sync_from_device()
print(f'end dotprod_{cu}')

print(f'start update_{cu}')
D.sync_to_device()
E.sync_to_device()
update.call(D,E,F)
F.sync_from_device()
print(f'end update_{cu}')



start dotprod_0


RuntimeError: Buffer submit failed: -22